# Homework 7

## Due Thursday, November 29th 2018 at 11:59 PM.

### Be sure to push the final version of your notebook to your GitHub repo.  Follow the instructions on the course website.

### Topics
####  [Part 1](#part_1):  Database schema [15 points]
* [Problem 1](#p1.1). Schema [15 points]

####  [Part 2](#part_2):  Insert records [35 points]
* [Problem 2](#p2.1). Baseline model [15 points]
* [Problem 3](#p2.2). Reduced model [10 points]
* [Problem 4](#p2.3). L1 penalty model [10 points]

####  [Part 3](#part_3):  Queries [20 pts]
* [Problem 5](#p3.1). Best model coefficients [10 points]
* [Problem 6](#p3.2). Best model score [10 points]

---

<a id='part_1'></a>
# Part 1:  Database schema

<a id='p1.1'></a>
## Problem 1 (15 points): 

In this problem you will set up a SQL database using the `sqllite` package in Python. The purpose of the database will be to store parameters and model results related to a simple *Logistic Regression* problem. Rather than keeping the results in `Numpy` arrays as we usually do, the idea here is to make use of a `SQL` database to materialize the results so that it can easily be accessed from disk at a later stage.

The design of the database should be flexible enough so that the results from different model iterations can be stored in the database. It should also be able to deal with a different set of features by model iteration.

A list of the tables to include in the database and the relevant fields in each table is shown below (tables are in bold):

**model_params**: 
* id 
* desc 
* param_name
* value

**model_coeffs**
* id 
* desc 
* feature_name
* value

**model_results**
* id 
* desc 
* train_score
* test_score

Create a `SQL` database called `regression.sqlite` containing the three tables shown above.

In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('regression.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS model_params")
cursor.execute("DROP TABLE IF EXISTS model_coeffs")
cursor.execute("DROP TABLE IF EXISTS model_results")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE model_params (
               id INTEGER, 
               desc TEXT, 
               param_name TEXT, 
               value TEXT)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE model_coeffs (
               id INTEGER, 
               desc TEXT, 
               feature_name TEXT, 
               value FLOAT)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE model_results (
               id INTEGER, 
               desc TEXT, 
               train_score FLOAT, 
               test_score FLOAT)''')

db.commit() # Commit changes to the database

<a id='part_2'></a>
# Part 2: Insert records

In this section you will populate the database you created in the previous question with some records for a number of different model iterations / scenarios.

<a id='p2.1'></a>
## Problem 2 (15 points): 
Create a baseline Logistic Regression model using the provided code (below).  Insert the relevant arrays into the corresponding tables in the database.

**model_params**
Values from the [`get_params`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.get_params) method.

**model_coeffs**
Coefficient and intercept values of the fitted model (see `coef_` and `intercept_` attributes in the documentation).

**model_results**
Train and validation accuracy obtained from the [`score`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score) method.


#### Remarks
* Reference scikit-learn documentation to get more detail on the methods / attributes list above:  
[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

* Note that the *id* and *desc* are just identifier fields used to identify the results from a specific model iteration or scenario. For example for the baseline model you could set *id = 1* and *desc = "Baseline model"*.


#### Suggestions
You may want to create a function to save data to the database.  You will be able to re-use this function in subsequent sections.

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer

%matplotlib inline

/Users/zhengyang/anaconda/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [3]:
# Load data
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

In [4]:
# Fit model
clf = LogisticRegression()
clf.fit(X_train, y_train)

/Users/zhengyang/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [5]:
params = {'C': '1.0', 'class_weight': 'None', 'dual': 'False', 'fit_intercept': 'True',
          'intercept_scaling': '1', 'max_iter': '100', 'multi_class':'warn',
          'n_jobs': 'None', 'penalty': 'l2', 'random_state': 'None', 'solver': 'warn',
          'tol': '0.0001', 'verbose': '0', 'warm_start': 'False'}

In [6]:
feature_names = X.columns

In [7]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

In [9]:
def insert_to_params(Id, desc, params):
    for k, v in params.items():
        vals_to_insert = (Id, desc, k, v)
        cursor.execute('''INSERT INTO model_params 
                  (id, desc, param_name, value)
                  VALUES (?, ?, ?, ?)''', vals_to_insert)

In [10]:
def insert_to_coeffs(Id, desc, feature_names, coef_values, intercept):
    for i in range(len(feature_names)):
        vals_to_insert = (Id, desc, feature_names[i], coef_values[i])
        cursor.execute('''INSERT INTO model_coeffs 
                  (id, desc, feature_name, value)
                  VALUES (?, ?, ?, ?)''', vals_to_insert)
    vals_to_insert = (Id, desc, 'intercept', intercept)
    cursor.execute('''INSERT INTO model_coeffs 
              (id, desc, feature_name, value) 
              VALUES (?, ?, ?, ?)''', vals_to_insert)

In [11]:
def insert_to_model_results(Id, desc, train_score, test_score):
    vals_to_insert = (Id, desc, train_score, test_score)
    cursor.execute('''INSERT INTO model_results 
              (id, desc, train_score, test_score)
              VALUES (?, ?, ?, ?)''', vals_to_insert)

In [12]:
insert_to_params(1, 'Baseline', params)

In [13]:
insert_to_coeffs(1, 'Baseline', feature_names, clf.coef_[0], clf.intercept_[0])

In [14]:
insert_to_model_results(1, 'Baseline', train_score, test_score)

In [15]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [16]:
params_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_params)")]
query = '''SELECT * FROM model_params'''
viz_tables(params_cols, query)

,desc,id,param_name,value
0,Baseline,1,C,1.0
1,Baseline,1,class_weight,None
2,Baseline,1,dual,False
3,Baseline,1,fit_intercept,True
4,Baseline,1,intercept_scaling,1
5,Baseline,1,max_iter,100
6,Baseline,1,multi_class,warn
7,Baseline,1,n_jobs,None
8,Baseline,1,penalty,l2
9,Baseline,1,random_state,None


In [17]:
coeffs_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_coeffs)")]
query = '''SELECT * FROM model_coeffs'''
viz_tables(coeffs_cols, query)

,desc,feature_name,id,value
0,Baseline,mean radius,1,2.143352
1,Baseline,mean texture,1,0.073687
2,Baseline,mean perimeter,1,-0.148922
3,Baseline,mean area,1,0.015650
4,Baseline,mean smoothness,1,-0.104633
5,Baseline,mean compactness,1,-0.407477
6,Baseline,mean concavity,1,-0.594942
7,Baseline,mean concave points,1,-0.263499
8,Baseline,mean symmetry,1,-0.155283
9,Baseline,mean fractal dimension,1,-0.028102


In [18]:
results_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_results)")]
query = '''SELECT * FROM model_results'''
viz_tables(results_cols, query)

,desc,id,test_score,train_score
0,Baseline,1,0.938596,0.96044


<a id='p2.2'></a>
## Problem 3 (10 points): 
Create a second model using only the features included in the list below (in `feature_cols`).  Insert the relevant arrays into the corresponding tables in the database.

Remember to update the `id` and `desc` values for the second iteration.

#### Suggestions
* Name this second model `"Reduced model"`.

In [19]:
feature_cols = ['mean radius',
                'texture error',
                'worst radius',
                'worst compactness',
                'worst concavity']

In [20]:
X = pd.DataFrame(data.data, columns=data.feature_names)
red_X = X.loc[:,feature_cols]

# Split into train and test
red_X_train, red_X_test, y_train, y_test = train_test_split(red_X, y, test_size=0.2, random_state=87)

In [21]:
# Fit model
clf = LogisticRegression()
clf.fit(red_X_train, y_train)

/Users/zhengyang/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [22]:
y_train_pred = clf.predict(red_X_train)
y_test_pred = clf.predict(red_X_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

In [23]:
params = {'C': '1.0', 'class_weight': 'None', 'dual': 'False', 'fit_intercept': 'True',
          'intercept_scaling': '1', 'max_iter': '100', 'multi_class':'warn',
          'n_jobs': 'None', 'penalty': 'l2', 'random_state': 'None', 'solver': 'warn',
          'tol': '0.0001', 'verbose': '0', 'warm_start': 'False'}

In [24]:
feature_names = red_X.columns

In [25]:
insert_to_params(2, 'Reduced model', params)
insert_to_coeffs(2, 'Reduced model', feature_names, clf.coef_[0], clf.intercept_[0])
insert_to_model_results(2, 'Reduced model', train_score, test_score)

In [26]:
params_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_params)")]
query = '''SELECT * FROM model_params'''
viz_tables(params_cols, query)

,desc,id,param_name,value
0,Baseline,1,C,1.0
1,Baseline,1,class_weight,None
2,Baseline,1,dual,False
3,Baseline,1,fit_intercept,True
4,Baseline,1,intercept_scaling,1
5,Baseline,1,max_iter,100
6,Baseline,1,multi_class,warn
7,Baseline,1,n_jobs,None
8,Baseline,1,penalty,l2
9,Baseline,1,random_state,None


In [27]:
coeffs_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_coeffs)")]
query = '''SELECT * FROM model_coeffs'''
viz_tables(coeffs_cols, query)

,desc,feature_name,id,value
0,Baseline,mean radius,1,2.143352
1,Baseline,mean texture,1,0.073687
2,Baseline,mean perimeter,1,-0.148922
3,Baseline,mean area,1,0.015650
4,Baseline,mean smoothness,1,-0.104633
5,Baseline,mean compactness,1,-0.407477
6,Baseline,mean concavity,1,-0.594942
7,Baseline,mean concave points,1,-0.263499
8,Baseline,mean symmetry,1,-0.155283
9,Baseline,mean fractal dimension,1,-0.028102


In [28]:
results_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_results)")]
query = '''SELECT * FROM model_results'''
viz_tables(results_cols, query)

,desc,id,test_score,train_score
0,Baseline,1,0.938596,0.960440
1,Reduced model,2,0.885965,0.945055


<a id='p2.3'></a>
## Problem 4 (10 points): 
Create one last model using an **l1-penalty** ($L_{1}$) term and **all** the features. Insert the relevant arrays into the corresponding tables in the database.

**Hint:** Refer to the `penalty` parameter of the `LogisticRegression` class.

#### Suggestions
Call this model `"L1 penalty model"`.

In [29]:
# Fit model
clf = LogisticRegression(penalty = 'l1')
clf.fit(X_train, y_train)

/Users/zhengyang/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [30]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

In [31]:
params = {'C': '1.0', 'class_weight': 'None', 'dual': 'False', 'fit_intercept': 'True',
          'intercept_scaling': '1', 'max_iter': '100', 'multi_class':'warn',
          'n_jobs': 'None', 'penalty': 'l1', 'random_state': 'None', 'solver': 'warn',
          'tol': '0.0001', 'verbose': '0', 'warm_start': 'False'}

In [32]:
insert_to_params(3, 'L1 penalty model', params)
insert_to_coeffs(3, 'L1 penalty model', X.columns, clf.coef_[0], clf.intercept_[0])
insert_to_model_results(3, 'L1 penalty model', train_score, test_score)

In [33]:
params_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_params)")]
query = '''SELECT * FROM model_params'''
viz_tables(params_cols, query)

,desc,id,param_name,value
0,Baseline,1,C,1.0
1,Baseline,1,class_weight,None
2,Baseline,1,dual,False
3,Baseline,1,fit_intercept,True
4,Baseline,1,intercept_scaling,1
5,Baseline,1,max_iter,100
6,Baseline,1,multi_class,warn
7,Baseline,1,n_jobs,None
8,Baseline,1,penalty,l2
9,Baseline,1,random_state,None


In [34]:
coeffs_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_coeffs)")]
query = '''SELECT * FROM model_coeffs'''
viz_tables(coeffs_cols, query)

,desc,feature_name,id,value
0,Baseline,mean radius,1,2.143352
1,Baseline,mean texture,1,0.073687
2,Baseline,mean perimeter,1,-0.148922
3,Baseline,mean area,1,0.015650
4,Baseline,mean smoothness,1,-0.104633
5,Baseline,mean compactness,1,-0.407477
6,Baseline,mean concavity,1,-0.594942
7,Baseline,mean concave points,1,-0.263499
8,Baseline,mean symmetry,1,-0.155283
9,Baseline,mean fractal dimension,1,-0.028102


In [35]:
results_cols = [col[1] for col in cursor.execute("PRAGMA table_info(model_results)")]
query = '''SELECT * FROM model_results'''
viz_tables(results_cols, query)

,desc,id,test_score,train_score
0,Baseline,1,0.938596,0.960440
1,Reduced model,2,0.885965,0.945055
2,L1 penalty model,3,0.947368,0.964835


<a id='part_3'></a>

# Part 3:  Queries

<a id='p3.1'></a>
## Problem 5 (10 points): 
Query the database to identify the model with the highest validation score.
* Print the id of the best model and the corresponding validation score.
  ```bash
  Best model id: 
  Best validation score:
  ```
* Print the feature names and corresponding coefficients of that model.

In [36]:
query = '''SELECT id, max(test_score) FROM model_results'''
query_res = cursor.execute(query).fetchall()
print('Best model id:', query_res[0][0])
print('Best validation score: %.2f' %query_res[0][1])

Best model id: 3
Best validation score: 0.95


In [37]:
query = '''SELECT feature_name, value FROM model_coeffs WHERE id = 3'''
cursor.execute(query).fetchall()

[('mean radius', 5.92679597833741),
 ('mean texture', 0.016851680046292016),
 ('mean perimeter', -0.4779272537290069),
 ('mean area', -2.9795010724416965e-05),
 ('mean smoothness', 0.0),
 ('mean compactness', 0.0),
 ('mean concavity', 0.0),
 ('mean concave points', 0.0),
 ('mean symmetry', 0.0),
 ('mean fractal dimension', 0.0),
 ('radius error', 0.0),
 ('texture error', 0.0),
 ('perimeter error', 0.6762094758743072),
 ('area error', -0.05249040510086294),
 ('smoothness error', 0.0),
 ('compactness error', 0.0),
 ('concavity error', 0.0),
 ('concave points error', 0.0),
 ('symmetry error', 0.0),
 ('fractal dimension error', 0.0),
 ('worst radius', 0.0),
 ('worst texture', -0.2063807634632699),
 ('worst perimeter', -0.10419082787997846),
 ('worst area', -0.02801941961457824),
 ('worst smoothness', 0.0),
 ('worst compactness', 0.0),
 ('worst concavity', -2.263407727105091),
 ('worst concave points', 0.0),
 ('worst symmetry', 0.0),
 ('worst fractal dimension', 0.0),
 ('intercept', 0.0)]

<a id='p3.2'></a>
## Problem 6 (10 points): 

Use the coefficients extracted in the previous question to reproduce the validation score (accuracy) of the best performing model (as stored in the database).

**Hint:** You should be able to achieve this by overwriting the relevant variables in the Logistic regression object, i.e. there is no need write your own formula to generate individual predictions (you are welcome to do this if you want).

#### Remarks
The problem demos a simple scenario in which someone with access to your database can easily reproduce your results.

In [45]:
query = '''SELECT feature_name, value FROM model_coeffs where id = 3'''
res = cursor.execute(query).fetchall()

coeffs = np.zeros((1, len(res)-1))
intercept = np.zeros((1, 1))

for i, (coeff_name, value) in enumerate(res[:-1]):
    coeffs[0, i] = value
    
intercept[0, 0] = res[-1][1]

clf2 = LogisticRegression()
clf2.coef_, clf2.intercept_, clf2.classes_ = coeffs, intercept, np.unique(y)
print('Best validation score: %.2f' % clf2.score(X_test, y_test))

Best validation score: 0.95
